In [21]:
import os
import re

from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI

def load_text_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

class AITutor:
    """
    A class that facilitates a back-and-forth conversation between a student and an AI tutor.
    It uses an LLM (Large Language Model) to guide students through questions, provide feedback, 
    and help them understand science concepts without directly giving them the answers.

    This class is designed to work with any LLM that has a chat function.

    Attributes:
        llm (Any LLM with a chat function): The LLM model used to interact with the student.
        message_history (list): A history of messages exchanged between the student and the tutor.
    
    Methods:
        initiate_conversation(grade, topic): Initiates the tutoring session by asking the student for more details.
        get_response(student_input): Handles student input and provides a response using the LLM.
        get_message_history(): Returns the history of messages in the conversation.
    """

    def __init__(self, llm_model, instructions, display_system=False):
        self.llm = llm_model
        self.instructions = instructions
        self.message_history = []

        # Add these as a "system prompt"
        self.system_prompt = f"""
{instructions}

## Your Task

You are a tutor for science students in grades 6-8. 
Following the instructions above, provide supportive assistance to the student user.
        """
        if display_system:
            print(self.system_prompt)

        # Append initial request from AI tutor
        self.initiate_conversation()

    def initiate_conversation(self):
        """
        Initiates the conversation with the student, asking for the grade level and topic they are working on.
        """
        init_message = f"""
Hi! I'm here to help you with your science questions. 

I won't do the work for you, but I'll guide you through each step so you can understand and feel more confident.

To start, what grade are you in and what do you need help with?
        """
        # Initialize the conversation with the system prompt
        self.message_history.append(ChatMessage(role="system", content=self.system_prompt))
        # Append initial request from AI tutor
        self.message_history.append(ChatMessage(role="assistant", content=init_message))

    def get_response(self, student_input):
        """
        Handles student input and provides a response.
        The LLM will respond based on the system instructions, conversation history, and the input provided by the student.
        """
        # Add the student's message to the history
        self.message_history.append(ChatMessage(role="user", content=student_input))
        
        # Get the response from the LLM
        response = self.llm.chat(self.message_history).message.content
        
        # Add the AI's response to the history
        self.message_history.append(ChatMessage(role="assistant", content=response))
        
        return response

class AIStudent:
    """
    A class that simulates a student interacting with an AI tutor.
    It uses an LLM (Large Language Model) to generate student responses based on the tutor's guidance.
    
    This class is designed to work with any LLM that has a chat function.

    Attributes:
        llm (Any LLM with a chat function): The LLM model used to impersonate the student.
        message_history (list): A history of messages exchanged between the student and the tutor.
        personality (str): Describes the student's behavior and learning style during the session.

    Methods:
        respond_to_tutor(tutor_input): Generates a student response to the tutor's input.
        get_message_history(): Returns the history of messages in the conversation.
    """

    def __init__(self, llm_model, personality="curious", display_system=False):
        self.llm = llm_model
        self.message_history = []
        self.personality = personality

        # Add system prompt to guide the LLM in generating responses that mimic a student
        self.system_prompt = f"""
You are a grade 6-8 science student who {self.personality}
Your RESPONSES SHOULD BE VERY CONCISE.
Your RESPONSES SHOULD REFLECT THE LIMITED KNOWLEDGE of a grade 6-8 student.
        """
        if display_system:
            print(self.system_prompt)

        # Initialize the conversation with the system prompt
        self.message_history.append(ChatMessage(role="system", content=self.system_prompt))

    def respond_to_tutor(self, tutor_input):
        """
        Handles tutor input and generates a student response using the LLM.
        The LLM will respond based on the student personality, conversation history, and the input provided by the tutor.
        """
        # Add the tutor's message to the history
        self.message_history.append(ChatMessage(role="assistant", content=tutor_input))

        # Generate the student's response from the LLM
        student_response = self.llm.chat(self.message_history).message.content
        
        # Add the student's response to the history
        self.message_history.append(ChatMessage(role="user", content=student_response))

        return student_response

    def get_message_history(self):
        """
        Returns the full conversation history between the student and the tutor.
        """
        return self.message_history


# Load OpenAI API key
openai_api_key = load_text_file(file_path='key.txt').strip("\n")
# Initialize the tutor with the LLM and instructions
instructions = load_text_file('science_tutor/tutor_instructions.txt')
llm_model = OpenAI(model="gpt-4o-mini", api_key=openai_api_key)

def get_init_prompt(problem):
    return f'Can you help me solve this problem: "{problem}"'
    
problems = [
    'What are the three main states of matter, and how do particles behave in each state?',
    'What is the difference between potential energy and kinetic energy? Can you give an example of each?',
    'What are the functions of plant cell parts like the cell wall, chloroplasts, and vacuoles?',
    'How do ecosystems maintain balance, and what happens if a species is removed?',
    'How does the water cycle work, and what role do evaporation and condensation play?',
    'What causes the phases of the Moon, and why do we only see one side of the Moon from Earth?',
    'How does gravity affect the motion of objects in space and on Earth?',
    'What are the key differences between physical and chemical changes? Can you provide examples of each?',
    'How do renewable and nonrenewable energy sources impact the environment?',
    'What is photosynthesis, and why is it important for both plants and animals?',
    'How do sound waves travel, and how does the medium affect their speed?',
    'Why are some materials better conductors of electricity than others?',
    'How do magnets work, and what happens when opposite poles of two magnets come near each other?',
    'What is the difference between an element, a compound, and a mixture? Can you give examples?',
    'How does the human body respond to exercise, and what systems are involved in that response?'
]

problems = [
    'What is the law of conservation of mass, and how does it apply to chemical reactions?',
    'Why do we experience different seasons on Earth, and how is this related to the tilt of Earth\'s axis?',
    'What are the layers of Earth\'s atmosphere, and what happens in each layer?',
    'How do plate tectonics cause earthquakes and the formation of mountains?',
    'What is the difference between inherited traits and learned behaviors in living organisms?',
    'How does a simple electric circuit work, and what components are necessary to make it function?',
    'What is an atom, and how are protons, neutrons, and electrons arranged within it?',
    'Why is biodiversity important in an ecosystem, and how does it affect the stability of the environment?',
    'How do vaccines help protect the human body from diseases?',
    'What are fossils, and how do they provide evidence for the theory of evolution?',
    'How do weathering and erosion shape Earth\'s surface over time?',
    'What is the difference between a meteor, a meteoroid, and a meteorite?',
    'How do enzymes function in the human body, and why are they important for digestion?',
    'What are acids and bases, and how can you test the pH of a substance?',
    'How does sound differ from light in the way it travels, and why can light travel through space but sound cannot?'
]

personalities = {
    'curious': 'loves asking questions and eagerly explores new topics to deepen their understanding.',
    'easily distracted': 'does not stay focused and frequently drifts off-task during learning sessions.',
    'anxious': 'feels nervous about getting things wrong and hesitates before attempting answers.',
    'enthusiastic': 'engages with excitement, showing a genuine interest in learning and actively participating in discussions.',
    'answer-seeking': 'does not provide input and only tries to get the tutor to give them the answers directly instead of working through the problem.'
}

In [22]:
import random
import json

def run_conversation(display=False, num_iters=3, personality=None, problem=None, student_last=True):
    if problem is None:
        # Select a random problem and personality
        problem = random.choice(problems)
    if personality is None:
        personality = random.choice(list(personalities.keys()))

    # Initialize tutor
    tutor_chatbot = AITutor(llm_model, instructions)

    # Initialize the student with the selected random personality
    ai_student = AIStudent(llm_model, personality=personalities[personality], display_system=False)

    
    # Initiate the conversation with the problem
    init_prompt = get_init_prompt(problem)
    ai_student.message_history.append(ChatMessage(role="assistant", content=init_prompt))
    tutor_response = tutor_chatbot.get_response(init_prompt)
    if display:
        print(f"Personality: {personality}")
        print(f"\nStudent: {init_prompt}")
        print(f"\nTutor: {tutor_response}")

    # Run a conversation
    for i in range(num_iters-2):
        # Student responds to the tutor
        student_response = ai_student.respond_to_tutor(tutor_response)
        
        # Tutor provides a new response to the student's input
        tutor_response = tutor_chatbot.get_response(student_response)
        if display:
            print(f"\nStudent: {student_response}")
            print(f"\nTutor: {tutor_response}")
    if student_last:
        # Student responds to the tutor
        student_response = ai_student.respond_to_tutor(tutor_response)
        tutor_chatbot.message_history.append(ChatMessage(role="user", content=student_response))
    return tutor_chatbot.message_history

# Function to convert chat history to a list of dictionaries and save to JSON
def write_chat_history_to_json(chat_history, file_name):
    chat_history_dict = [{'role': message.role, 'content': message.content} for message in chat_history]
    
    with open(file_name, 'w') as file:
        json.dump(chat_history_dict, file, indent=4)

# Function to load chat history from JSON and convert back to ChatMessage objects
def read_chat_history_from_json(file_name):
    with open(file_name, 'r') as file:
        chat_history_dict = json.load(file)
    
    chat_history = [ChatMessage(role=message['role'], content=message['content']) for message in chat_history_dict]
    return chat_history

start_num = 16

# Run and save the conversation histories to separate text files
for i, problem in enumerate(problems):
    # Run the conversation
    message_history = run_conversation(num_iters=2, personality='answer-seeking', problem=problem)
    
    # Write the conversation to a text file
    file_name = f"data/tutor_interactions/conversation_{i+start_num}.json"
    write_chat_history_to_json(message_history, file_name)

    print(f"Conversation {i+start_num} saved to {file_name}")

    if (i+start_num)==4:
        break

Conversation 16 saved to data/tutor_interactions/conversation_16.json
Conversation 17 saved to data/tutor_interactions/conversation_17.json
Conversation 18 saved to data/tutor_interactions/conversation_18.json
Conversation 19 saved to data/tutor_interactions/conversation_19.json
Conversation 20 saved to data/tutor_interactions/conversation_20.json
Conversation 21 saved to data/tutor_interactions/conversation_21.json
Conversation 22 saved to data/tutor_interactions/conversation_22.json
Conversation 23 saved to data/tutor_interactions/conversation_23.json
Conversation 24 saved to data/tutor_interactions/conversation_24.json
Conversation 25 saved to data/tutor_interactions/conversation_25.json
Conversation 26 saved to data/tutor_interactions/conversation_26.json
Conversation 27 saved to data/tutor_interactions/conversation_27.json
Conversation 28 saved to data/tutor_interactions/conversation_28.json
Conversation 29 saved to data/tutor_interactions/conversation_29.json
Conversation 30 save